In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(0)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
#TEST NOTES:  First attempt on new training schedule functionality.
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'bw2color_WganUnetSched'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

c_lr=1e-3
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/3
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

x_tfms = [BlackAndWhiteTransform()]
torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=1, self_attention=True).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=256, scale=32, self_attention=True).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
trainer = WGANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e5)])
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [6]:
scheds=[]
scheds.extend(WGANTrainSchedule.generate_schedules(szs=[64,128,256], bss=[64,16,4], path=IMAGENET, x_tfms=x_tfms, keep_pcts=[0.5,0.2,0.1], save_base_name=proj_id, 
        critic_lrs_start=c_lrs, gen_lrs_start=g_lrs, lrs_change_factor=0.5, random_seed=42))

In [ ]:
netG.freeze_to(1)
trainer.train(scheds=scheds)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")
/media/jason/Projects/Deep Learning/Deep Learning Projects/Jupyter Experiments/1_BlackWhiteToColor/fasterai/generators.py:130: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h<target_h or w<target_w:


Failed to generate graph for model: %43 : Long() = onnx::Constant[value={0}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400
Failed to generate graph for model: %465 : Long() = onnx::Constant[value={2}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400
  0%|          | 0/19819 [00:00<?, ?it/s]

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


  0%|          | 20/19819 [00:24<2:46:03,  1.99it/s]
WDist -55.40673065185547; RScore 123.07626342773438; FScore 67.6695327758789; GAddlLoss 7897.1298828125; Iters: 20; GCost: -27.250226974487305; EpsPenalty: 1.514776587486267
  0%|          | 40/19819 [00:34<2:38:42,  2.08it/s]
WDist -87.92462921142578; RScore 173.63954162597656; FScore 85.71491241455078; GAddlLoss 7621.279296875; Iters: 40; GCost: -76.99665832519531; EpsPenalty: 3.015068769454956
  0%|          | 60/19819 [01:01<3:44:54,  1.46it/s]
WDist -48.70841979980469; RScore 168.2924041748047; FScore 119.583984375; GAddlLoss 7509.36181640625; Iters: 60; GCost: -108.79192352294922; EpsPenalty: 2.832233428955078
  0%|          | 80/19819 [01:11<2:37:09,  2.09it/s]
WDist -1.8136138916015625; RScore 182.35531616210938; FScore 180.5417022705078; GAddlLoss 6518.28369140625; Iters: 80; GCost: -180.62881469726562; EpsPenalty: 3.325345993041992
  1%|          | 100/19819 [01:22<2:38:36,  2.07it/s]
WDist -2.725982666015625; RScore 179.84